Keras - Autoencoder

In [29]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yfin

yfin.pdr_override()

# Dow Jones 30
symbols_table = pd.read_html("https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Components",header=0)[1]
symbols = list(symbols_table.loc[:, "Symbol"])
index_symbol = ['^DJI']


# Dates
start_date = '2008-01-01'
end_date = '2017-12-31'


# Download the data
data = pd.DataFrame()

for i in range(len(symbols)):
    symbols[i]=symbols[i].replace(u'\xa0',u'').replace("NYSE:","")

symbols.remove('DOW') # DOW data are unvailable on yahoo

for i in range(len(symbols)):
    print('Downloading.... ', i, symbols[i])

    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    data[symbols[i]] = pdr.get_data_yahoo(symbols[i], start_date, end_date)['Adj Close']
    data_index = pdr.get_data_yahoo(index_symbol, start_date, end_date)['Adj Close']

# Remove the missing the data from the dataframe
data = data.dropna()
data_index = data_index.dropna()

# Save the data
data.to_csv('dj30_10y.csv', sep=',', encoding='utf-8')
data_index.to_csv('dj30_index_10y.csv', sep=',', encoding='utf-8')

print(data.head())

Downloading....  0 MMM
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Downloading....  1 AXP
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Downloading....  2 AMGN
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Downloading....  3 AAPL
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Downloading....  4 BA
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Downloading....  5 CAT
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Downloading....  6 CVX
[*********

In [36]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Load data
data = pd.read_csv('dj30_10y.csv', sep=',', engine='python')
assets = data.columns.values[1:].tolist()
data = data.iloc[:, 1:]

# Load index
index = pd.read_csv('dj30_index_10y.csv', sep=',', engine='python')
index = index.iloc[-data.values.shape[0]:, 1:]

# Normalize data
scaler = MinMaxScaler([0.1,0.9])
data_X = scaler.fit_transform(data)
scaler_index = MinMaxScaler([0.1,0.9])
index = scaler_index.fit_transform(index)

# Number of components
N_COMPONENTS = 3

## Autoencoder - Keras
# Network hyperparameters
n_inputs = len(assets)
n_core = N_COMPONENTS
n_outputs = n_inputs

# Create model
input = Input(shape=(n_inputs,))
# Encoder
encoded = Dense(n_core, activation='sigmoid')(input)
# Decoder
decoded = Dense(n_outputs, activation='sigmoid')(encoded)

# define model
autoencoder = Model(input, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Testing in-sample
X_train = data_X
X_test = data_X

# Training parameters
epochs = 20

# Fit the model
history = autoencoder.fit(X_train,\
                          X_train,\
                          epochs=epochs,\
                          batch_size=1,\
                          shuffle=True,\
                          verbose=1)

# Make AE predictions
y_pred_AE_keras = autoencoder.predict(X_test)

print('test loss: '+str(autoencoder.evaluate(y_pred_AE_keras, X_test)))


Epoch 1/20
2465/2465 [==============================] - 2s 711us/step - loss: 0.0389
Epoch 2/20
2465/2465 [==============================] - 2s 894us/step - loss: 0.0132
Epoch 3/20
2465/2465 [==============================] - 2s 701us/step - loss: 0.0064
Epoch 4/20
2465/2465 [==============================] - 2s 661us/step - loss: 0.0044
Epoch 5/20
2465/2465 [==============================] - 2s 729us/step - loss: 0.0036
Epoch 6/20
2465/2465 [==============================] - 2s 643us/step - loss: 0.0032
Epoch 7/20
2465/2465 [==============================] - 2s 646us/step - loss: 0.0027
Epoch 8/20
2465/2465 [==============================] - 2s 720us/step - loss: 0.0024
Epoch 9/20
2465/2465 [==============================] - 2s 703us/step - loss: 0.0022
Epoch 10/20
2465/2465 [==============================] - 1s 596us/step - loss: 0.0022
Epoch 11/20
2465/2465 [==============================] - 2s 642us/step - loss: 0.0021
Epoch 12/20
2465/2465 [==============================] - 2s 792

Tensorflow - Autoencoder

In [40]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
from sklearn.utils import shuffle

n_inputs = len(assets)
n_core = N_COMPONENTS
n_outputs = n_inputs

initializer = tf.initializers.glorot_normal()
w1 = tf.Variable(initializer([n_inputs, n_core]))
w2 = tf.transpose(w1)
b1 = tf.Variable(tf.zeros([n_core]))
b2 = tf.Variable(tf.zeros([n_outputs]))

Instructions for updating:
non-resource variables are not supported in the long term


In [41]:
# Building the encoder
def encoder(x):
    return tf.nn.sigmoid(tf.add(tf.matmul(x, w1), b1))

# Building the decoder
def decoder(x):
    return tf.nn.sigmoid(tf.add(tf.matmul(x, w2), b2))

In [42]:
X = tf.placeholder("float", [None, n_inputs])
Y = tf.placeholder("float", [None, n_inputs])
# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
# Prediction
y_pred = decoder_op
y_true = X


# Training parameters
lr = 0.01
epochs = 20
batch_size = 1


mse = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.AdamOptimizer(lr).minimize(mse)

# Start Training
# Start a new TF session
with tf.Session() as sess:
    # Initialize the network
    sess.run(tf.global_variables_initializer())

    # Training
    for i in range(epochs):
        X_train1 = shuffle(X_train)
        for j in range(X_train.shape[0] // batch_size):
            batch_y = X_train1[j * batch_size:j * batch_size + batch_size, :]
            batch_x = X_train1[j * batch_size:j * batch_size + batch_size, :]
            _, loss_value = sess.run([optimizer, mse], feed_dict={X: batch_x, Y: batch_y})

        # Display loss
        print('Epoch: %i -> Loss: %f' % (i, loss_value))

    # Make predictions
    y_pred_AE_tf = sess.run(decoder_op, feed_dict={X: X_train, Y: X_train})
    print('Test Error: %f' % tf.losses.mean_squared_error(X_train, y_pred_AE_tf).eval())


Epoch: 0 -> Loss: 0.001723
Epoch: 1 -> Loss: 0.019032
Epoch: 2 -> Loss: 0.003953
Epoch: 3 -> Loss: 0.000775
Epoch: 4 -> Loss: 0.000909
Epoch: 5 -> Loss: 0.004756
Epoch: 6 -> Loss: 0.002089
Epoch: 7 -> Loss: 0.005746
Epoch: 8 -> Loss: 0.002506
Epoch: 9 -> Loss: 0.002924
Epoch: 10 -> Loss: 0.001526
Epoch: 11 -> Loss: 0.001658
Epoch: 12 -> Loss: 0.001932
Epoch: 13 -> Loss: 0.002264
Epoch: 14 -> Loss: 0.003534
Epoch: 15 -> Loss: 0.000872
Epoch: 16 -> Loss: 0.003574
Epoch: 17 -> Loss: 0.001141
Epoch: 18 -> Loss: 0.001429
Epoch: 19 -> Loss: 0.001274
Test Error: 0.002522
